In [8]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [1]:
import os
os.chdir('../Dataset/Train')
os.getcwd()

'/Users/benedettosimone/PycharmProjects/LipNetProve/Dataset/Train'

In [2]:
data=[]
for l in os.listdir():
    os.chdir(l)
    for m in os.listdir():
        data.append([m, l])
    os.chdir('..')

In [4]:
data

[['F01_words08_06.jpg', 'r'],
 ['F09_words08_07.jpg', 'r'],
 ['M07_words08_02.jpg', 'r'],
 ['M01_words08_05.jpg', 'r'],
 ['F08_words08_03.jpg', 'r'],
 ['F07_words08_01.jpg', 'r'],
 ['M01_words08_10.jpg', 'r'],
 ['M01_words08_04.jpg', 'r'],
 ['F08_words08_02.jpg', 'r'],
 ['M07_words08_03.jpg', 'r'],
 ['F09_words08_06.jpg', 'r'],
 ['M08_words08_01.jpg', 'r'],
 ['F01_words08_07.jpg', 'r'],
 ['F09_words08_04.jpg', 'r'],
 ['F01_words08_05.jpg', 'r'],
 ['M08_words08_03.jpg', 'r'],
 ['F09_words08_10.jpg', 'r'],
 ['M07_words08_01.jpg', 'r'],
 ['M01_words08_06.jpg', 'r'],
 ['F07_words08_02.jpg', 'r'],
 ['F07_words08_03.jpg', 'r'],
 ['F08_words08_01.jpg', 'r'],
 ['M01_words08_07.jpg', 'r'],
 ['F01_words08_04.jpg', 'r'],
 ['M08_words08_02.jpg', 'r'],
 ['F09_words08_05.jpg', 'r'],
 ['F01_words08_10.jpg', 'r'],
 ['F09_words08_01.jpg', 'r'],
 ['M08_words08_06.jpg', 'r'],
 ['M07_words08_04.jpg', 'r'],
 ['M07_words08_10.jpg', 'r'],
 ['M01_words08_03.jpg', 'r'],
 ['F08_words08_05.jpg', 'r'],
 ['F07_wor

In [14]:
os.getcwd()

'/Users/benedettosimone/PycharmProjects/LipNetProve/codes'

In [6]:
import csv
os.chdir('../../codes')
header = ['filename', 'label']

with open('training_labels.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

In [10]:
train_data = pd.read_csv('training_labels.csv')
Y = train_data[['label']]

kf = KFold(n_splits = 5)

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True)

In [18]:
idg = ImageDataGenerator(rescale=1./255)

In [12]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [15]:
def create_new_model():
    import os
    import numpy as np
    from keras.models import Sequential # To initialise the nn as a sequence of layers
    from keras.layers import Convolution2D # To make the convolution layer for 2D images
    from keras.layers import MaxPooling2D #
    from keras.layers import Flatten
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.callbacks import CSVLogger
    from tensorflow.keras.optimizers import RMSprop
    from keras.layers import BatchNormalization
    from tensorflow.keras.optimizers import Adam
    from keras.models import load_model
    from keras.callbacks import ModelCheckpoint
    from keras.layers import Activation
    from keras.layers import GlobalAveragePooling2D

    csv=CSVLogger("2_adam-Copy1.log")
    #filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


    # Initialising the CNN
    classifier = Sequential()

    # Step 1 - Convolution
    classifier.add(Convolution2D(32,(3,3),input_shape = (224,224,1), activation = 'sigmoid',name='convo2'))
    # Step 2 - Pooling
    classifier.add(MaxPooling2D(pool_size = (2,2)))

    # Step 1 - Convolution
    classifier.add(Convolution2D(32,(3,3),activation = 'sigmoid',name='convo3'))
    # Step 2 - Pooling
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    # Step 3 - Flattening
    classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo4'))
    # Step 2 - Pooling
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    # Step 3 - Flatten
    classifier.add(Flatten())

    classifier.add(Dropout((0.6)))
    classifier.add(Dense(1024))
    classifier.add(BatchNormalization(scale = False))
    classifier.add(Activation('relu'))
    classifier.add(Dropout((0.4)))
    classifier.add(Dense(20, activation = 'softmax'))

In [19]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = '/saved_models/'
fold_var = 1

for train_index, val_index in kf.split(np.zeros(2585),Y):
	training_data = train_data.iloc[train_index]
	validation_data = train_data.iloc[val_index]

	train_data_generator = idg.flow_from_dataframe(training_data, directory = '../Dataset/Train',
						       x_col = "filename", y_col = "label",
						       class_mode = "categorical", shuffle = True)
	valid_data_generator  = idg.flow_from_dataframe(validation_data, directory =  '../Dataset/Train',
							x_col = "filename", y_col = "label",
							class_mode = "categorical", shuffle = True)

	# CREATE NEW MODEL
	model = create_new_model()
	# COMPILE NEW MODEL
	model.compile(loss='categorical_crossentropy',
		      optimizer="Adam",
		      metrics=['accuracy'])

	# CREATE CALLBACKS
	checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var),
							monitor='val_accuracy', verbose=1,
							save_best_only=True, mode='max')
	callbacks_list = [checkpoint]
	# There can be other callbacks, but just showing one because it involves the model name
	# This saves the best model
	# FIT THE MODEL
	history = model.fit(train_data_generator,
			    epochs=10,
			    callbacks=callbacks_list,
			    validation_data=valid_data_generator)
	#PLOT HISTORY
	#		:
	#		:

	# LOAD BEST MODEL to evaluate the performance of the model
	model.load_weights("/saved_models/model_"+str(fold_var)+".h5")

	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))

	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])

	tf.keras.backend.clear_session()

	fold_var += 1

/Users/benedettosimone/PycharmProjects/LipNetProve/venv/lib/python3.9/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 2068 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/Users/benedettosimone/PycharmProjects/LipNetProve/venv/lib/python3.9/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 517 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
2022-01-12 12:24:22.011052: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


AttributeError: 'NoneType' object has no attribute 'compile'